In [1]:
import joblib
import sys
import torch
import math
import numpy as np
# from tqdm.notebook import tqdm
from tqdm import tqdm
import os
import pandas as pd
from sktime.datasets import load_from_tsfile_to_dataframe

import warnings
warnings.filterwarnings("ignore")

### Prepare the UEA classification datasets

In [2]:
dataset_names=[
    'ArticularyWordRecognition', 
    'AtrialFibrillation', 
    'BasicMotions', 
    'CharacterTrajectories', 
    'Cricket', 
    'DuckDuckGeese', 
    'ERing', 
    'EigenWorms', 
    'Epilepsy', 
    'EthanolConcentration', 
    'FaceDetection', 
    'FingerMovements', 
    'HandMovementDirection', 
    'Handwriting', 
    'Heartbeat', 
    # 'InsectWingbeat',
    'JapaneseVowels', 
    'LSST', 
    'Libras', 
    'MotorImagery', 
    'NATOPS', 
    'PEMS-SF', 
    'PenDigits', 
    'PhonemeSpectra', 
    'RacketSports', 
    'SelfRegulationSCP1', 
    'SelfRegulationSCP2', 
    'SpokenArabicDigits', 
    'StandWalkJump', 
    'UWaveGestureLibrary'
]


# Change the dataset root to the path of your dataset
dataset_root='/data/wanghaokai/VQShape-main/Multivariate_ts'
save_root = '/data/wanghaokai/VQShape-main/uea'

def interpolate_uts(x, new_len):
    x = torch.from_numpy(x)
    x = torch.nn.functional.interpolate(x.view(1, 1, -1), new_len, mode='linear')
    return x.squeeze()

def dataframe_to_list(df):
    """
    Convert each element of a pandas DataFrame to a list.

    :param df: pandas DataFrame.
    :return: List containing all elements of the DataFrame.
    """
    elements_list = []
    for _, row in df.iterrows():
        elements_list.extend(row.tolist())
    return elements_list

def load_single(dataset_root, dataset):
    df, labels = load_from_tsfile_to_dataframe(f"{dataset_root}/{dataset}/{dataset}_{flag}.ts")
    data_list = dataframe_to_list(df)
    df = None
    data_list = [interpolate_uts(x.values.flatten(), new_len=512).float() for x in data_list]
    # data_list = [(x - np.mean(x))/(np.std(x) + np.finfo(float).eps) for x in data_list]
    data_list = [(x - x.mean()) / (x.var() + 1e-5).sqrt() for x in data_list]
    return torch.stack(data_list, dim=0).numpy()


# Load the UEA datasets and write each univariate TS into a csv file
for flag in ['TRAIN', 'TEST']:
    for dataset in dataset_names:
        print(dataset, end=": ")
        save_dir = f"{save_root}/{flag}/{dataset}"
        os.makedirs(save_dir, exist_ok=True)
        data = load_single(dataset_root, dataset)
        print(data.shape)

        for i, x in enumerate(tqdm(data)):
            df = pd.DataFrame(x)
            df.to_csv(f"{save_dir}/{i}.csv", index=False, header=False)

ArticularyWordRecognition: (2475, 512)


100%|██████████| 2475/2475 [00:04<00:00, 501.13it/s]


AtrialFibrillation: (30, 512)


100%|██████████| 30/30 [00:00<00:00, 497.36it/s]


BasicMotions: (240, 512)


...

100%|██████████| 28587/28587 [00:37<00:00, 772.06it/s] 


StandWalkJump: (60, 512)


100%|██████████| 60/60 [00:00<00:00, 451.93it/s]


UWaveGestureLibrary: (960, 512)


100%|██████████| 960/960 [00:01<00:00, 800.29it/s]


### Prepare the Forecasting datasets

In [7]:
import pandas as pd
import glob

glob.glob("../data/PILE/forecasting/autoformer/*")

['../data/PILE/forecasting/autoformer/weather.csv',
 '../data/PILE/forecasting/autoformer/ETTm2.csv',
 '../data/PILE/forecasting/autoformer/ETTh1.csv',
 '../data/PILE/forecasting/autoformer/electricity.csv',
 '../data/PILE/forecasting/autoformer/national_illness.csv',
 '../data/PILE/forecasting/autoformer/traffic.csv',
 '../data/PILE/forecasting/autoformer/exchange_rate.csv',
 '../data/PILE/forecasting/autoformer/ETTh2.csv',
 '../data/PILE/forecasting/autoformer/ETTm1.csv']

In [22]:
from tqdm import tqdm
import os
import numpy as np

dataset = 'traffic'
df = pd.read_csv(f'/data/wanghaokai/Time-Series-Library/dataset/traffic/{dataset}.csv')
save_dir = f'/data/wanghaokai/RQ-VAE-Project/forecasting/TEST/{dataset}'
test_size = 0.2
step_size = 48
seq_lengths = [512] # [96, 192, 336, 512]
data = df.values[:, 1:]
data = data[int(data.shape[0]*(1-test_size)):, :]
start_idx = np.arange(0, data.shape[0] - step_size, step_size)

if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

for l in seq_lengths:
    for c in tqdm(range(data.shape[1]), desc=f'{l}'):
        for start in start_idx:
            x = data[start:start+l, c]
            pd.DataFrame(x).to_csv(f"{save_dir}/{c}_{start}_{l}.csv", index=False, header=False)


512: 100%|██████████| 862/862 [00:50<00:00, 17.04it/s]


In [18]:
from tqdm import tqdm
import os
import numpy as np
import pandas as pd

dataset = 'ETTh2'
df = pd.read_csv(f'/data/wanghaokai/Time-Series-Library/dataset/ETT-small/{dataset}.csv')
save_dir = f'/data/wanghaokai/RQ-VAE-Project/forecasting/TEST/{dataset}'

step_size = 24
seq_lengths = [512] # [96, 192, 336, 512]
data = df.values[:, 1:]
data = data[12 * 30 * 24 + 4 * 30 * 24:, :]
print(data.shape)
start_idx = np.arange(0, data.shape[0] - step_size, step_size)

if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

for l in seq_lengths:
    for c in tqdm(range(data.shape[1]), desc=f'{l}'):
        for start in start_idx:
            x = data[start:start+l, c]
            pd.DataFrame(x).to_csv(f"{save_dir}/{c}_{start}_{l}.csv", index=False, header=False)


(5900, 7)


512: 100%|██████████| 7/7 [00:01<00:00,  3.59it/s]


In [19]:
from tqdm import tqdm
import os
import numpy as np

dataset = 'national_illness'
df = pd.read_csv(f'/data/wanghaokai/Time-Series-Library/dataset/illness/{dataset}.csv')
save_dir = f'/data/wanghaokai/RQ-VAE-Project/forecasting/TEST/{dataset}'
test_size = 0.2
step_size = 12
seq_lengths = [24, 36, 48, 60]
data = df.values[:, 1:]
data = data[int(data.shape[0]*(1-test_size)):, :]
start_idx = np.arange(0, data.shape[0] - step_size, step_size)

if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

for l in seq_lengths:
    for c in tqdm(range(data.shape[1]), desc=f'{l}'):
        for start in start_idx:
            x = data[start:start+l, c]
            pd.DataFrame(x).to_csv(f"{save_dir}/{c}_{start}_{l}.csv", index=False, header=False)


60: 100%|██████████| 7/7 [00:00<00:00, 116.63it/s]


### Prepare the UCR classification datasets


In [11]:
from sktime.datasets import load_from_ucr_tsv_to_dataframe
import os
from tqdm import tqdm
import numpy as np


def df_to_feature(df, labels):
    features = []
    for _, row in df.iterrows():
        sample = []
        for c in row.tolist():
            # sample.append(interpolate_uts(c.values.flatten(), seq_len))
            sample.append(c.values.flatten())
        features.append(np.stack(sample, axis=0))
    features = np.stack(features, axis=0)

    labels = pd.Series(labels, dtype="category")
    labels = pd.DataFrame(labels.cat.codes, dtype=np.int8).values

    return features, labels


root = f"../data/timeseries_lib/UCR_2018"
datasets = os.listdir(root)
datasets = sorted([d for d in datasets if not d.startswith(".")])

count = 0

for d in tqdm(datasets):
    save_dir = f'../data/VQShape/ucr/TEST/{d}'
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)

    features, labels = load_from_ucr_tsv_to_dataframe(f"{root}/{d}/{d}_TEST.tsv")
    features, labels = df_to_feature(features, labels)

    for i in range(features.shape[0]):
        x = features[i].flatten()
        pd.DataFrame(x).to_csv(f"{save_dir}/{i}.csv", index=False, header=False)
        count += 1

count

100%|██████████| 128/128 [01:01<00:00,  2.09it/s]


130603

In [2]:
import pandas as pd
import torch
import torch.nn as nn
# import plotly.express as px
# import plotly.graph_objects as go  # 用于多序列对比可视化
from sklearn.preprocessing import MinMaxScaler

# 定义移动平均模块
class moving_avg(nn.Module):
    """移动平均块：提取时间序列趋势项"""
    def __init__(self, kernel_size, stride):
        super(moving_avg, self).__init__()
        self.kernel_size = kernel_size
        self.avg = nn.AvgPool1d(kernel_size=kernel_size, stride=stride, padding=0)

    def forward(self, x):
        # 两端填充，保证输出长度与输入一致
        front = x[:, 0:1, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        end = x[:, -1:, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        x = torch.cat([front, x, end], dim=1)
        # 适配AvgPool1d维度要求 (bs, channels, seq_len)
        x = x.permute(0, 2, 1)
        x = self.avg(x)
        x = x.permute(0, 2, 1)
        return x

# 定义序列分解模块
class series_decomp(nn.Module):
    """序列分解块：分解为残差项（高频）和趋势项（低频）"""
    def __init__(self, kernel_size):
        super(series_decomp, self).__init__()
        self.moving_avg = moving_avg(kernel_size, stride=1)

    def forward(self, x):
        moving_mean = self.moving_avg(x)
        res = x - moving_mean
        return res, moving_mean

In [5]:
from tqdm import tqdm
import os
import numpy as np

dataset = 'traffic'
df = pd.read_csv(f'/data/wanghaokai/Time-Series-Library/dataset/traffic/{dataset}.csv')
save_dir = f'/data/wanghaokai/RQ-VAE-Project/forecasting_96/TRAIN/{dataset}'
test_size = 0.2
step_size = 24
seq_lengths = [96] # [96, 192, 336, 512]
data = df.values[:, 1:]
data = data[:int(data.shape[0]*(1-test_size)), :]
start_idx = np.arange(0, data.shape[0] - step_size, step_size)

# kernel_size = 13  # 可根据数据趋势调整（比如电力日周期用7，周周期用15）
# decomp = series_decomp(kernel_size=kernel_size)

if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

for l in seq_lengths:
    for c in tqdm(range(data.shape[1]), desc=f'{l}'):
        for start in start_idx:
            x = data[start:start+l, c]
            pd.DataFrame(x).to_csv(f"{save_dir}/{c}_{start}_{l}.csv", index=False, header=False)


96: 100%|██████████| 862/862 [04:46<00:00,  3.00it/s]


In [6]:
from tqdm import tqdm
import os
import numpy as np

dataset = 'weather'
df = pd.read_csv(f'/data/wanghaokai/Time-Series-Library/dataset/weather/{dataset}.csv')
save_dir = f'/data/wanghaokai/RQ-VAE-Project/forecasting_96/TRAIN/{dataset}'
test_size = 0.2
step_size = 24
seq_lengths = [96] # [96, 192, 336, 512]
data = df.values[:, 1:]
data = data[:int(data.shape[0]*(1-test_size)), :]
start_idx = np.arange(0, data.shape[0] - step_size, step_size)

# kernel_size = 13  # 可根据数据趋势调整（比如电力日周期用7，周周期用15）
# decomp = series_decomp(kernel_size=kernel_size)

if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

for l in seq_lengths:
    for c in tqdm(range(data.shape[1]), desc=f'{l}'):
        for start in start_idx:
            x = data[start:start+l, c]
            pd.DataFrame(x).to_csv(f"{save_dir}/{c}_{start}_{l}.csv", index=False, header=False)


96: 100%|██████████| 21/21 [00:21<00:00,  1.04s/it]


In [7]:
from tqdm import tqdm
import os
import numpy as np

dataset = 'weather'
df = pd.read_csv(f'/data/wanghaokai/Time-Series-Library/dataset/weather/{dataset}.csv')
save_dir = f'/data/wanghaokai/RQ-VAE-Project/forecasting_96/TRAIN/{dataset}'
test_size = 0.2
step_size = 96
seq_lengths = [96] # [96, 192, 336, 512]
data = df.values[:, 1:]
data = data[:int(data.shape[0]*(1-test_size)), :]
start_idx = np.arange(0, data.shape[0] - step_size, step_size)

# kernel_size = 13  # 可根据数据趋势调整（比如电力日周期用7，周周期用15）
# decomp = series_decomp(kernel_size=kernel_size)

if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

for l in seq_lengths:
    for c in tqdm(range(data.shape[1]), desc=f'{l}'):
        for start in start_idx:
            x = data[start:start+l, c]
            pd.DataFrame(x).to_csv(f"{save_dir}/{c}_{start}_{l}.csv", index=False, header=False)


96: 100%|██████████| 21/21 [00:04<00:00,  4.62it/s]
